Este notebook é um versão da análise deste artigo
http://trasel.com.br/usando-sql-lite-para-encontrar-os-desertos-de-noticias-no-brasil/
usando Pandas.

Bibliotecas a serem utilizadas:

In [22]:
import ftplib
import os
import sqlite3
import zipfile

import pandas as pd
import requests

In [13]:
os.makedirs('inputs', exist_ok=True)
with requests.get("https://github.com/voltdatalab/Atlas-Analytics/raw/master/db/atlas.db") as r:
    with open('inputs/atlas.db', 'wb') as f:
        f.write(r.content)

ftp://geoftp.ibge.gov.br/organizacao_do_territorio/estrutura_territorial/divisao_territorial/2015/dtb_2015.zip

In [4]:
municipio_zip = os.path.join('inputs', 'dtb_2015.zip')

In [ ]:
ftp = ftplib.FTP("geoftp.ibge.gov.br")
ftp.login()
with open(municipio_zip, 'wb') as f:
    ftp.cwd("organizacao_do_territorio/estrutura_territorial/divisao_territorial/2015")
    ftp.retrbinary("RETR dtb_2015.zip", f.write)
ftp.close()

In [16]:
with zipfile.ZipFile(municipio_zip) as municipios:
    municipios.extract('dtb_2015/RELATORIO_DTB_BRASIL_MUNICIPIO.xls', path='inputs')

In [114]:
municipios_ibge = pd.read_excel('inputs/dtb_2015/RELATORIO_DTB_BRASIL_MUNICIPIO.xls',
                                index_col="Nome_Município")
municipios_ibge.index = municipios_ibge.index.str.upper()
municipios_ibge

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo
Nome_Município,,,,,,,,
ALTA FLORESTA D'OESTE,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015
ALTO ALEGRE DOS PARECIS,11,Rondônia,2,Leste Rondoniense,6,Cacoal,379,1100379
ALTO PARAÍSO,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,403,1100403
ALVORADA D'OESTE,11,Rondônia,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346
ARIQUEMES,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,23,1100023
BURITIS,11,Rondônia,1,Madeira-Guaporé,1,Porto Velho,452,1100452
CABIXI,11,Rondônia,2,Leste Rondoniense,8,Colorado do Oeste,31,1100031
CACAULÂNDIA,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,601,1100601
CACOAL,11,Rondônia,2,Leste Rondoniense,6,Cacoal,49,1100049


https://pandas.pydata.org/pandas-docs/stable/io.html#reading-tables
https://pandas.pydata.org/pandas-docs/stable/io.html#sqlite-fallback

In [128]:
cnx = sqlite3.connect('inputs/atlas.db')
atlas = pd.read_sql_query("SELECT * from atlas", cnx, index_col="cidade")
cnx.close()

In [129]:
joined = atlas.join(municipios_ibge)

In [133]:
# https://stackoverflow.com/a/14033137/2631881
missing = joined.iloc[pd.isnull(joined).any(1).nonzero()].index
missing

Index(['ALVORADA D`OESTE', 'AMAMBAÍ', 'AMAMBAÍ', 'ANTONIO PRADO', 'BIGUAÇÚ',
       'CAMPOS DE JORDÃO', 'ELIAS FAUSANTO', 'ESPIGÃO D`OESTE',
       'ESTANCIA VELHA', 'GETULIO VARGAS', 'HERVAL D`OESTE', 'HERVAL D`OESTE',
       'HERVAL D`OESTE', 'HERVAL D`OESTE', 'HERVAL D´OESTE', 'IJUI',
       'LUIS EDUARDO MAGALHÃES', 'LUIS EDUARDO MAGALHÃES',
       'LUIS EDUARDO MAGALHÃES', 'LUIS EDUARDO MAGALHÃES', 'MACAPA',
       'MOGI-MIRIM', 'MOGI-MIRIM', 'MOGI-MIRIM', 'NOVA VENÊCIA',
       'NÃO INFORMADO NO SITE DA ANJ', 'PARATI', 'PARATI', 'PASANTOS BONS',
       'PIÇARRAS', 'PIÇARRAS', 'PIÇARRAS', 'POXORÉO', 'RESTINGA SECA',
       'RESTINGA SECA', 'RESTINGA SECA', 'RESTINGA SECA', 'REVALDO',
       'SANTA BARBARA DO SUL', 'SANTA BÁRBARA D`OESTE',
       'SANTA BÁRBARA D`OESTE', 'SANTA BÁRBARA D`OESTE',
       'SANTA BÁRBARA D`OESTE', 'SANTA BÁRBARA D`OESTE',
       'SANTA BÁRBARA D`OESTE', 'SANTA BÁRBARA D`OESTE',
       'SANTANA DO LIVRAMENTO', 'SANTANA DO LIVRAMENTO',
       'SANTANA DO

In [135]:
atlas.loc[atlas.index.str.contains("AUGUSANTO")]

,id,meio,nome,regiao_metropolitana,uf,estado,regiao,pais,fonte
cidade,,,,,,,,,
SANTO AUGUSANTO,id5310,Jornal,O CELEIRO,n/d,RS,Rio Grande do Sul,Sul,Brasil,Adjori-Rs


In [123]:
municipios_ibge[municipios_ibge.index.str.contains("AUGUSTO")]

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo
Nome_Município,,,,,,,,
AUGUSTO CORRÊA,15,Pará,4,Nordeste Paraense,10,Bragantina,909,1500909
AUGUSTO SEVERO,24,Rio Grande do Norte,1,Oeste Potiguar,3,Médio Oeste,1305,2401305
AUGUSTO DE LIMA,31,Minas Gerais,6,Central Mineira,25,Curvelo,4809,3104809
AUGUSTO PESTANA,43,Rio Grande do Sul,1,Noroeste Rio-grandense,8,Ijuí,1503,4301503
SANTO AUGUSTO,43,Rio Grande do Sul,1,Noroeste Rio-grandense,8,Ijuí,17806,4317806


In [136]:
atlas.loc[missing]

,id,meio,nome,regiao_metropolitana,uf,estado,regiao,pais,fonte
cidade,,,,,,,,,
ALVORADA D`OESTE,id2314,Jornal,CORREIO DO VALE,n/d,RO,Rondônia,Norte,Brasil,Secom/PR
AMAMBAÍ,id1208,Jornal,CORREIO DA FRONTEIRA,n/d,MS,Mato Grosso do Sul,Centro-Oeste,Brasil,Secom/PR
AMAMBAÍ,id1209,Jornal,A GAZETA,n/d,MS,Mato Grosso do Sul,Centro-Oeste,Brasil,Secom/PR
AMAMBAÍ,id1208,Jornal,CORREIO DA FRONTEIRA,n/d,MS,Mato Grosso do Sul,Centro-Oeste,Brasil,Secom/PR
AMAMBAÍ,id1209,Jornal,A GAZETA,n/d,MS,Mato Grosso do Sul,Centro-Oeste,Brasil,Secom/PR
ANTONIO PRADO,id5157,Jornal,PANORAMA PRADENSE,n/d,RS,Rio Grande do Sul,Sul,Brasil,Adjori-Rs
BIGUAÇÚ,id5696,Jornal,BIGUAÇU EM FOCO,n/d,SC,Santa Catarina,Sul,Brasil,Central de Diários
CAMPOS DE JORDÃO,id5563,Jornal,JORNAL CAMPOS DE JORDÃO E CIA,n/d,SP,São Paulo,Sudeste,Brasil,Adjori-SP
ELIAS FAUSANTO,id4136,Jornal,FOLHA DE ELIAS FAUSTO,n/d,SP,São Paulo,Sudeste,Brasil,Secom/PR


In [138]:
municipios_ibge[municipios_ibge.index.str.contains("HERVAL")]

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo
Nome_Município,,,,,,,,
HERVAL D'OESTE,42,Santa Catarina,1,Oeste Catarinense,4,Joaçaba,6702,4206702
HERVAL,43,Rio Grande do Sul,7,Sudeste Rio-grandense,34,Jaguarão,7104,4307104
SANTA MARIA DO HERVAL,43,Rio Grande do Sul,5,Metropolitana de Porto Alegre,24,Gramado-Canela,16956,4316956
SÃO JOSÉ DO HERVAL,43,Rio Grande do Sul,1,Noroeste Rio-grandense,13,Soledade,18465,4318465
